<a href="https://colab.research.google.com/github/ShriMLEngineer/furniture_classification/blob/main/model_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from flask import Flask, request, jsonify
import numpy as np
from PIL import Image
import torch

app = Flask(__name__)

# Load the model
path = '/content/furniture_classification/detr-main'

model = torch.hub.load(path, 'detr_resnet50', source="local", pretrained=True, num_classes=4)
checkpoint = torch.load("/content/furniture_classification/outputs/checkpoint.pth", map_location='cpu')

model.load_state_dict(checkpoint['model'], strict=False)
model.eval();


# Define a route for image classification
@app.route('/predict', methods=['POST'])
def predict():
    # Get the image file from the request
    file = request.files['image']

    # Open the image file and preprocess it
    img = Image.open(file)
    img = img.resize((224, 224))
    img = np.array(img) / 255.0
    img = img.reshape(1, 224, 224, 3)

    # Use the model to make predictions
    preds = model.predict(img)

    # Convert the predictions to class labels
    classes = np.argmax(preds, axis=1)

    # Return the class label as a JSON response
    return jsonify({'class': int(classes[0])})

if __name__ == '__main__':
    app.run()